# Libraries

https://www.geeksforgeeks.org/python-urllib-module/
https://docs.python.org/3/library/urllib.robotparser.html
https://www.scraperapi.com/web-scraping/scrapy/

In [1]:
import pandas as pd
from pymongo import MongoClient
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import time
from yaml import load, FullLoader
from urllib.robotparser import RobotFileParser

# connect to mongodb server

In [2]:
with open("my_data/mongodb.yml", 'r') as f:
    config = load(f, Loader=FullLoader)

In [3]:
username = config["username"]
password = config["password"]
cluster = config["cluster"]
database = config["cluster"]

client = MongoClient(f"mongodb+srv://{username}:{password}@{cluster}/?retryWrites=true&w=majority",
                     tls=True, tlsAllowInvalidCertificates=True)

In [4]:
db = client["companies_website"]

In [23]:
db.websites.find_one()

{'_id': ObjectId('67edaa801b2aead64a806219'),
 'domain': 'restrenacar.com',
 'index': 5472,
 'content': {'1': {'url': 'https://restrenacar.com/',
   'page': '<!DOCTYPE html>\n<html lang="es">\n\n    \n\n    <head>\n        <meta charset="UTF-8">\n        <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">\n        <title>\n            \nConcesionario Ocasión en Barcelona |\n Restrenacar | Concesionario Ocasión en Barcelona\n        </title>\n\n        <meta name="description" content="Concesionario Ocasión en Barcelona. Coches nuevos y ocasión. Compra y mantenimiento.  ¡Visítanos!">\n\n        <meta name="google-site-verification" content="zT7qWkWsFzhqh_pbgEFlpcsbIGEh8nAtJcqQthaNH6Q" />\n\n        <link rel="preload" href="/static/fonts/nunito/Nunito-Regular.bcf84d6cb8a0.woff2" as="font" type="font/woff2" crossorigin>\n        <link rel="preload" href="/static/fonts/nunito/Nunito-Bold.412073df419a.woff2" as="font" type="font/woff2" crossorigin>\n     

In [5]:
db = client["companies_website"]
sch = db.get_collection("website")
doc = sch.find_one()

None


# read data/urls to scrape

In [6]:
filepath = "results/searcher_4/analisis_searcher4.xlsx"
df = pd.read_excel(filepath)

In [7]:
df

,Unnamed: 0.1,order,query,company_name,title,snippet,link,display_link,matching,original,...,Web site,NACE,Employees,Date of Establishment,Operating revenue,Establishment Year,Ordered operating revenue,Size,NACE_SECTION,Original Company Name
0,0,0,Company Name-site,People Plus Innovation Sl,Cuidado | People Plus | Barcelona,Servicios de atención a domicilio. Gente Mayor...,https://www.socialpeopleplus.com/,www.socialpeopleplus.com,False,www.optimaenergy.net,...,www.optimaenergy.net,8899,485,2008-06-26 00:00:00,10869.475319,2008,1,Large,Q,People Plus Innovation S.L.
1,1,1,Company Name-site,People Plus Innovation Sl,PEOPLE PLUS INNOVATION SL | Datos de Empresa |...,Toda la información de PEOPLE PLUS INNOVATION ...,https://cincodias.elpais.com/directorio-empres...,cincodias.elpais.com,False,www.optimaenergy.net,...,www.optimaenergy.net,8899,485,2008-06-26 00:00:00,10869.475319,2008,1,Large,Q,People Plus Innovation S.L.
2,2,2,Company Name-site,People Plus Innovation Sl,Acusan una empresa de mentir para ganar el con...,"Jan 24, 2024 ... ... People Plus Innovation S....",https://www.heraldo.es/noticias/aragon/zaragoz...,www.heraldo.es,False,www.optimaenergy.net,...,www.optimaenergy.net,8899,485,2008-06-26 00:00:00,10869.475319,2008,1,Large,Q,People Plus Innovation S.L.
3,3,3,Company Name-site,People Plus Innovation Sl,El Gobierno de Zaragoza adjudica el nuevo cont...,"Sep 11, 2020 ... La adjudicación a las empresa...",https://www.zaragoza.es/sede/servicio/noticia/...,www.zaragoza.es,False,www.optimaenergy.net,...,www.optimaenergy.net,8899,485,2008-06-26 00:00:00,10869.475319,2008,1,Large,Q,People Plus Innovation S.L.
4,4,4,Company Name-site,People Plus Innovation Sl,Ayuntamiento de Logroño,"Jul 3, 2024 ... PEOPLE PLUS INNOVATION, S.L.U....",https://logrono.es/actas-mesa-de-contratacion?...,logrono.es,False,www.optimaenergy.net,...,www.optimaenergy.net,8899,485,2008-06-26 00:00:00,10869.475319,2008,1,Large,Q,People Plus Innovation S.L.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9948,9948,5,Company Name-site,Codebay Innovation,Futbol 7 y Futbol Sala La Amistad Tenerife - T...,"Jan 21, 2020 ... CODEBAY INNOVATION F7. Jugado...",https://futbol7amistad.mygol.es/tournaments/17...,futbol7amistad.mygol.es,False,www.codebay-innovation.com,...,www.codebay-innovation.com,6311,24,2010-08-24 00:00:00,NaN,2010,2,Small,J,Codebay Innovation SL.
9949,9949,6,Company Name-site,Codebay Innovation,COPA VETERANOS + ... - Futbol 7 y Futbol Sala ...,"Apr 23, 2024 ... CODEBAY INNOVATION F7. Jugado...",https://futbol7amistad.com/tournaments/103/mat...,futbol7amistad.com,False,www.codebay-innovation.com,...,www.codebay-innovation.com,6311,24,2010-08-24 00:00:00,NaN,2010,2,Small,J,Codebay Innovation SL.
9950,9950,7,Company Name-site,Codebay Innovation,"220 front end developer Jobs in Spain, March 2...",En Solusoft estamos buscando a gente como tú: ...,https://www.glassdoor.com/Job/spain-front-end-...,www.glassdoor.com,False,www.codebay-innovation.com,...,www.codebay-innovation.com,6311,24,2010-08-24 00:00:00,NaN,2010,2,Small,J,Codebay Innovation SL.
9951,9951,8,Company Name-site,Codebay Innovation,CODEBAY SOLUTIONS SL | Datos de Empresa | Cinc...,Toda la información de CODEBAY SOLUTIONS SL Co...,https://cincodias.elpais.com/directorio-empres...,cincodias.elpais.com,False,www.codebay-innovation.com,...,www.codebay-innovation.com,6311,24,2010-08-24 00:00:00,NaN,2010,2,Small,J,Codebay Innovation SL.


# clean urls

In [8]:
len(set(df["display_link"]))

5511

In [12]:
def load_blocklist(filepath = "my_data/blocklist_original.txt"):
    """
    Function to load the blocklist file where the domains and extensions to be excluded are stored.
    Args:
        - filepath (str): path to where the domains and extensions to be excluded are stored in a .txt file
    """
    with open(filepath, "r") as file:
        blocklist = [line.strip() for line in file if line.strip() and not line.startswith("#")]
    return blocklist


In [17]:
blocklist = load_blocklist()
display_links = df["display_link"]
websites = set(display_links) - set(blocklist)
len(websites)

5464

In [18]:
new_websites = set()
for i, row in df.iterrows():
    if row["display_link"] in websites and "instagram" not in row["link"] and "video" not in row["link"]:
        new_websites.add(row["link"])
        

In [19]:
len(new_websites)

8782

# scraper

In [74]:
def check_robots_txt(url:str)->bool:
    """
    Function to check if scraping is allowed according to robots.txt
    Args:
        - url (str): website of a possibly searched company
    Returns:
        - boolean value, indicating wheter it is possible the scraping (True) or not (False)
    
    """
    try:
        # Extract the domain to build the URL for robots.txt
        domain = re.search(r'^(https?://[^/]+)', url).group(0)
        robots_url = f"{domain}/robots.txt"
        
        # Create a RobotFileParser object and read the robots.txt
        rp = RobotFileParser()
        rp.set_url(robots_url)
        rp.read()
        
        
        # Check if the path is allowed
        if rp.can_fetch("*", url):
            print(f"Scraping is allowed for {url} based on robots.txt.")
            return True
        else:
            print(f"Scraping is not allowed for {url} based on robots.txt.")
            return False
    except Exception as e:
        print(f"Error checking robots.txt: {e}")
        return False

def scrape_website(url:str):
    """
    Function to scrape website and parse content. It checks the robots.txt first
    to know if scraping is allowed.
    Args:
        - url (str): website of a possibly searched company
    Returns
        - soup object with the result
    """
    
    # Check if scraping is allowed first
    if not check_robots_txt(url):
        print(f"Skipping scraping for {url} due to robots.txt restrictions.")
        return
    
    try:
        # Send an HTTP request to the website and if not raise HTTPError for bad responses
        response = requests.get(url)
        response.raise_for_status()  
        
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract title 
        title = soup.title.string if soup.title else 'No title found'
        print(f"Title of the page: {title}")
        
        # Extract all links (anchor tags)
        links = soup.find_all('a', href=True)
        print(f"Found {len(links)} links on the page:")
        for link in links[:5]: 
            print(f"Link: {link['href']}")
        
    except requests.exceptions.RequestException as e:
        # Capture errors like connection errors, timeout, etc.
        print(f"Request failed for {url}: {e}")
        return
    except Exception as e:
        # Catch any other unexpected errors
        print(f"An error occurred while scraping {url}: {e}")
        return
    return soup


def scrape_website(url):
    """
    Function to scrape website and parse content.
    Args:
        - url (str): website of a possibly searched company
    Returns
        - soup
    """
    
    try:
        response = requests.get(url)
        response.raise_for_status() 
        
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract title 
        title = soup.title.string if soup.title else 'No title found'
        print(dir(soup))
        print(f"Title of the page: {title}")
        
        # Extract all links (anchor tags)
        links = soup.find_all('a', href=True)
        print(f"Found {len(links)} links on the page:")
        for link in links[:5]:  # Show first 5 links 
            print(f"Link: {link['href']}")
        
    except requests.exceptions.RequestException as e:
        # Capture errors like connection errors, timeout, etc.
        print(f"Request failed for {url}: {e}")
        return
    except Exception as e:
        # Catch any other unexpected errors
        print(f"An error occurred while scraping {url}: {e}")
        return
    return soup


In [75]:

url = "https://www.rayomajadahonda.com" # www.rayomajadahonda.com 
url = "https://librebor.me/en/borme/empresa/mercadona/.html"
url = "https://www.mercadona.es/"
res = scrape_website(url)

# Adding a small delay between requests to avoid overloading the server
time.sleep(2)


['ASCII_SPACES', 'DEFAULT_BUILDER_FEATURES', 'EMPTY_ELEMENT_EVENT', 'END_ELEMENT_EVENT', 'MAIN_CONTENT_STRING_TYPES', 'ROOT_TAG_NAME', 'START_ELEMENT_EVENT', 'STRING_ELEMENT_EVENT', '_TreeTraversalEvent', '__annotations__', '__bool__', '__call__', '__class__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__unicode__', '__weakref__', '_all_strings', '_decode_markup', '_event_stream', '_feed', '_find_all', '_find_one', '_format_tag', '_indent_string', '_insert', '_is_xml', '_lastRecursiveChild', '_last_descendant', '_linkage_fixer', '_markup_is_url', '_markup_resem

In [ ]:
BeautifulSoup(res,lmx)

In [88]:
print(res.prettify())

<!DOCTYPE html>
<html lang="es">
 <head>
  <meta charset="utf-8"/>
  <link href="/manifest.json" rel="manifest"/>
  <link href="/favicon.ico" rel="shortcut icon"/>
  <link href="https://www.mercadona.es/" rel="canonical">
   <meta content="width=device-width,initial-scale=1,shrink-to-fit=no" name="viewport"/>
   <meta content="ie=edge" http-equiv="X-UA-Compatible"/>
   <meta content="#000000" name="theme-color"/>
   <meta content="notranslate" name="google"/>
   <meta content="summary_large_image" name="twitter:card"/>
   <meta content="@Mercadona" name="twitter:site"/>
   <meta content="@Mercadona" name="twitter:creator"/>
   <meta content="https://mercadona.es" property="og:url"/>
   <meta content="Mercadona" property="og:title"/>
   <meta content="Empieza tu compra online en Mercadona, encuentra nuestros supermercados mÃ¡s cercanos o descubre cÃ³mo trabajar con nosotros" property="og:description"/>
   <meta content="https://mercadona.es/web-display-img.png" property="og:image"/>
   

# improved scraper

In [45]:
class ScrapeDomain():
    ##################
    # init
    ##################
    def __init__(self, domain:str, index:str, lang_codes:list=[], max_pages:int=1, accept_subdomains:bool=False, protocol:str='https://'):
        '''
        Initializer method
        Args:
            - domain: it is the website domain to be scraped
            - index: it is the identifier of that URL
            - protocol: 
            - lang_codes: list of languages that should be prioritazed when scraping
            - max_pages:
            - accept_subdomains:
            - protocol:            
        '''
        self.protocol = protocol
        self.domain = domain.lower().strip().replace('http://','').replace('https://','').replace('www.','')
        self.domain_link = self.protocol+self.domain
        self.json = {'domain': self.domain,
                      'index': index,
                     'content': {}} # All scraped pages are embedded documents within content dict
        self.max_pages = max_pages
        self.link_set = {self.domain_link}
        self.num_pages = 0
        self.scraped = set()
        self.accept_subdomains = accept_subdomains
        self.lang_codes = lang_codes
        if self.lang_codes:
            lang_idents = []
            for language in self.lang_codes:
                lang_idents.append("\/{}\/".format(language))
                lang_idents.append("\/{}-{}\/".format(language, language))
                lang_idents.append("\?lang={}".format(language))
                lang_idents.append("/{}$".format(language))
            self.lang_patterns = re.compile('|'.join(lang_idents), re.IGNORECASE)

    
    ##################
    # Utility functions
    ##################
    # Exclude downloadable files, pictures, etc from being scraped
    # List taken from ARGUS by datawizard1337 (I added xls and xlsx)
    # https://github.com/datawizard1337/ARGUS --> language prioritising was also inspired by ARGUS
    filetypes = set(['mng', 'pct', 'bmp', 'gif', 'jpg', 'jpeg', 'png', 'pst', 'psp', 'tif', 'tiff', 'ai', 'drw', 'dxf', 'eps', 'ps', 'svg',
            'mp3', 'wma', 'ogg', 'wav', 'ra', 'aac', 'mid', 'au', 'aiff',
            '3gp', 'asf', 'asx', 'avi', 'mov', 'mp4', 'mpg', 'qt', 'rm', 'swf', 'wmv', 'm4a',
            'css', 'pdf', 'doc', 'exe', 'bin', 'rss', 'zip', 'rar', 'msu', 'flv', 'dmg', 'xls', 'xlsx',
            'mng?download=true', 'pct?download=true', 'bmp?download=true', 'gif?download=true', 'jpg?download=true', 'jpeg?download=true', 'png?download=true', 'pst?download=true', 'psp?download=true', 'tif?download=true', 'tiff?download=true', 'ai?download=true', 'drw?download=true', 'dxf?download=true', 'eps?download=true', 'ps?download=true', 'svg?download=true',
            'mp3?download=true', 'wma?download=true', 'ogg?download=true', 'wav?download=true', 'ra?download=true', 'aac?download=true', 'mid?download=true', 'au?download=true', 'aiff?download=true',
            '3gp?download=true', 'asf?download=true', 'asx?download=true', 'avi?download=true', 'mov?download=true', 'mp4?download=true', 'mpg?download=true', 'qt?download=true', 'rm?download=true', 'swf?download=true', 'wmv?download=true', 'm4a?download=true',
            'css?download=true', 'pdf?download=true', 'doc?download=true', 'exe?download=true', 'bin?download=true', 'rss?download=true', 'zip?download=true', 'rar?download=true', 'msu?download=true', 'flv?download=true', 'dmg?download=true'])
    filetypes_pattern = '|'.join(['\.'+filetype+'$' for filetype in filetypes])

    def not_scraped(self):
        '''
        Returns the a list with URLs that have not been scraped. Uses synonims by:
            - setting http/http addresses as same urls
            - slash in the end is taking into account (http://mecklenburgische.de vs http://mecklenburgische.de/)
            - websites with .html are usually just synomims for the same url without that ending
        '''
        return list(self.link_set.difference(self.scraped))
    
    def det_prio(self):
        '''Determines which URL should be scraped next'''
        not_scraped = self.not_scraped()
        if not self.lang_codes:
            link_stack = sorted(not_scraped, key=len)
        else:
            correct_lang = []
            other_lang = []
            for link in not_scraped:
                if re.search(self.lang_patterns, link):
                    correct_lang.append(link)
                else:
                    other_lang.append(link)
            # Sort urls that were not yet scraped by link length
            link_stack = sorted(correct_lang, key=len) + sorted(other_lang, key=len)
        self.to_scrape = link_stack[0]
        
    def extractLinks(self):
        '''
        Parse html code to find links.
        Calls get_internalURL to test whether the link should be scraped.
        '''
        soup = BeautifulSoup(self.website.text,"html.parser")
        for a in soup.find_all('a', href=True):
           url = self.get_internalURL(a['href'])
           if url:
               self.link_set.add(self.standardize_link(url))
               
    def get_internalURL(self, url):
        '''
        Ignores javascript, mailto and telephone links as well as unwanted file endings
        '''
        pattern = re.compile(''.join(["^mailto:|^tel:|^javascript:|", self.filetypes_pattern]), re.IGNORECASE)
        if url and not re.search(pattern, url):
            if self.domain in url:
                if self.accept_subdomains == False:
                    # Test whether link doesn't contain a subdomain
                    cleaned_url = url.lower().replace('http://','').replace('https://','').replace('www.','')
                    if cleaned_url.split('.')[0]==self.domain.split('.')[0]:
                        return url
                else:
                    return url
            elif url[0:2]=="./":
                return self.domain_link+url.replace('./','/')
            elif url[0]=="/":
                return self.domain_link+url
            elif "http" not in url:
                return self.domain_link+'/'+url
                
    def standardize_link(self, url):
        '''Standardize the url to exclude duplicates'''
        # Define protocol
        url = url.replace('http://','').replace('https://','').replace('www.','')
        url = self.protocol+url
        # URL endings
        # First removes standalone slashes at the end, then removes # urls
        url = re.sub('#[^!\/]*$', '', re.sub('\/$', '', url))
        # Removes index.html
        url = re.sub('\/index\.html?\/?$', '', url)
        return url

    ##################
    # Scraper
    ##################
    def url_scraping(self, user_agent:str, collectionName:str, timeOutConnect:int=10, timeOutRead:int=15, timeBetweenRequests:int=2, check_robots:bool = False):
        '''
        Scrapes link
        Args:
            - userAgent: the name of the robot (should be changed to the name of your organization and the purpose of scraping)
            - collectionName (pymongo.synchronous.collection.Collection): value after dot can be changed and it refers to the database collection e.g. database.myfirstcollection, database.wpc_20200301
            - max_pages: maximum number of pages on domain to be scraped
            - preferred_langs: list of ISO language codes that should be prioritised while scraping (can be omitted)
            - accept_subdomains: if set to True, also allow the crawler to scrape subdomains, which have the format: subdomain.domain --> eg. https://www-genesis.destatis.de/genesis/online, so "www-genesis" is a subdomain of destatis.de (this is where you can accesss the database of destatis)
            - timeBetweenRequests: set the time between requests - in seconds (suggested 3-5 seconds).
            - timeOutConnect: maximum time in seconds to connect to the website
            - timeOutRead: maximum time in seconds to read the website  
            - check_robots: if set to True, it will read the robots.txt and respect the website policy

        '''
        self.det_prio()
        print('Scraping', self.to_scrape)
        headers = {'user-agent': user_agent}
        self.scraped.add(self.to_scrape)
        try:
            if check_robots:
                if not self.check_robots(self.to_scrape):
                    print()
                    # it should add to the self.scraped or manage somehow

            # s = requests.Session()
            # a = requests.adapters.HTTPAdapter(pool_connections=100, pool_maxsize=100, max_retries=20, pool_block=False)
            # s.mount('https://', a)
            # s.mount('http://', a)
            self.website=requests.get(self.to_scrape.encode("utf-8"), headers=headers, timeout=(timeOutConnect,timeOutRead)) # , verify=True
            # if self.website.status_code == 200:
            # soup = BeautifulSoup(self.website.text, 'lxml')
            # soup = BeautifulSoup(self.website.text, 'html.parser')
            
            self.num_pages += 1
            self.scraped.add(self.website.url) # Add scraped url to scraped set (in case of redirect)
            print('Success in scraping page no', self.num_pages, 'of Domain:', self.domain, 'with code', self.website.status_code)
            self.json['content'].update({str(self.num_pages): {'url': self.website.url,
                                                               'page': self.website.text,
                                                               'date': str(datetime.now())}})

            ### more info ###

            """            
            # Parse the HTML content using BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Extract title 
            title = soup.title.string if soup.title else 'No title found'
            print(f"Title of the page: {title}")
            
            # Extract all links (anchor tags)
            links = soup.find_all('a', href=True)
            print(f"Found {len(links)} links on the page:")
            for link in links[:5]: 
                print(f"Link: {link['href']}")
            """
        
            # I use website.url to obtain the url that was actually scraped 
            # (different to original url in case of redirects)
            if self.num_pages < self.max_pages:
                self.extractLinks()
                print("extracting links")
                print(self.link_set)
            msg = "success"

            if self.num_pages >= self.max_pages or not self.link_set.difference(self.scraped):
                # Save json to MongoDB when max_pages is reached or no links are left for scraping

                print("TRYING TO STORE IN MONGODB")
                print(self.json)
                msg = "success + stored?"
                # try:
                #     result = collectionName.insert_one(self.json)
                #     print('Saved scraped pages to database')
                #     msg = str(self.website.status_code)+ '- Stored'

                # except Exception as e:
                #     print('Error while saving into database occurred')
                #     print(f'Error message: {type(e)}: {e}')
                #     msg = str(self.website.status_code)+'- Not stored'

        except requests.ConnectionError:
            print('Connection problems', self.to_scrape)
            msg = 'Connection problems'
        except requests.HTTPError:
            print('HTTP error', self.to_scrape)
            msg = 'HTTP error'
        except requests.Timeout:
            print('Timeout occurred', self.to_scrape)
            msg = 'Timeout occurred'
        except requests.TooManyRedirects:
            print('Too many redirects', self.to_scrape)
            msg = 'Too many redirects'
        except requests.RequestException:
            print('Request exception', self.to_scrape)
            msg = 'Request exception'
        # except requests.SSLError:
        #     print('Certificate verification error')
        except Exception as e:
            print('Error scraping', )
            print(f'Error while scraping: {self.to_scrape} --> {type(e)}: {e}')
            msg = e

        finally:
            # Even if the current page produced an error, continue scraping if there are still page links left
            if self.link_set.difference(self.scraped) and self.num_pages<self.max_pages:
                # N second delay on purpose
                time.sleep(timeBetweenRequests)
                yield from self.url_scraping(user_agent, collectionName, timeOutConnect, timeOutRead, timeBetweenRequests)
        yield msg, self.to_scrape
    def check_robots_txt(self, url:str) -> bool :
        '''
        Checks if scraping is allowing according to the robots.txt
        '''
        try:
            # Extract the domain to build the URL for robots.txt
            domain = re.search(r'^(https?://[^/]+)', url).group(0)
            robots_url = f"{domain}/robots.txt"
            
            # Create a RobotFileParser object and read the robots.txt
            rp = RobotFileParser()
            rp.set_url(robots_url)
            rp.read()
            
            
            # Check if the path is allowed
            if rp.can_fetch("*", url):
                print(f"Scraping is allowed for {url} based on robots.txt.")
                return True
            else:
                print(f"Scraping is not allowed for {url} based on robots.txt.")
                return False
        except Exception as e:
            print(f"Error checking robots.txt: {e}")
            return False


<>:63: SyntaxWarning: invalid escape sequence '\/'
<>:64: SyntaxWarning: invalid escape sequence '\/'
<>:65: SyntaxWarning: invalid escape sequence '\?'
<>:84: SyntaxWarning: invalid escape sequence '\.'
<>:151: SyntaxWarning: invalid escape sequence '\/'
<>:151: SyntaxWarning: invalid escape sequence '\/'
<>:153: SyntaxWarning: invalid escape sequence '\/'
<>:63: SyntaxWarning: invalid escape sequence '\/'
<>:64: SyntaxWarning: invalid escape sequence '\/'
<>:65: SyntaxWarning: invalid escape sequence '\?'
<>:84: SyntaxWarning: invalid escape sequence '\.'
<>:151: SyntaxWarning: invalid escape sequence '\/'
<>:151: SyntaxWarning: invalid escape sequence '\/'
<>:153: SyntaxWarning: invalid escape sequence '\/'
/tmp/ipykernel_31653/4221437485.py:63: SyntaxWarning: invalid escape sequence '\/'
  lang_idents.append("\/{}\/".format(language))
/tmp/ipykernel_31653/4221437485.py:64: SyntaxWarning: invalid escape sequence '\/'
  lang_idents.append("\/{}-{}\/".format(language, language))
/tmp/

In [48]:
# Database collection to be used
collectionName = db.websites

# Parameters for ScrapeDomain class
max_pages = 10
preferred_langs = ['es']
accept_subdomains = False

# Parameters for scraping
userAgent='python-app/0.1 experimental for investigation purposes'
timeBetweenRequests=2
timeOutConnect=10
timeOutRead=15


In [285]:
# Loop through domain urls to scrape
# the index variable can be used to save your own unique enterprise identifier with the scraped data
# Remark: whenever a domain is written to MongoDB, a unique identifier of that scraping attempt (or document) is created.

domains_list = []
resolution_list = []
for index, domain in enumerate(websites, 1):
    URLScraping = ScrapeDomain(domain = domain,
                               index = index,
                               max_pages = max_pages,
                               lang_codes = preferred_langs,
                               accept_subdomains = accept_subdomains)
    for res,web in URLScraping.url_scraping(userAgent, collectionName, timeOutConnect, timeOutRead, timeBetweenRequests, False):
        resolution_list.append(res)
        domains_list.append(web)

Scraping https://assets-us-01.kc-usercontent.com
Success in scraping page no 1 of Domain: assets-us-01.kc-usercontent.com with code 400
Saved scraped pages to database
Scraping https://dieselogasolina.com
Success in scraping page no 1 of Domain: dieselogasolina.com with code 200
Saved scraped pages to database
Scraping https://ferchau.ofertas-trabajo.infojobs.net
Success in scraping page no 1 of Domain: ferchau.ofertas-trabajo.infojobs.net with code 200
Saved scraped pages to database
Scraping https://easyworks.es
Success in scraping page no 1 of Domain: easyworks.es with code 200
Saved scraped pages to database
Scraping https://cambiago.es
Success in scraping page no 1 of Domain: cambiago.es with code 200
Saved scraped pages to database
Scraping https://fhalmeria.com
Connection problems https://fhalmeria.com
Scraping https://grupperemata.com
Success in scraping page no 1 of Domain: grupperemata.com with code 200
Saved scraped pages to database
Scraping https://revert.es
Success in scr

In [259]:
import pandas as pd

df = pd.DataFrame({
    'Domain': domains_list, 
    'Resolution': resolution_list 
})

df

,Domain,Resolution
0,https://assets-us-01.kc-usercontent.com,400Stored
1,https://dieselogasolina.com,200Stored
2,https://ferchau.ofertas-trabajo.infojobs.net,200Stored
3,https://easyworks.es,200Stored
4,https://cambiago.es,200Stored
5,https://fhalmeria.com,Connection problems
6,https://grupperemata.com,200Stored
7,https://revert.es,200Stored
8,https://careers.axa.com,200Stored


In [210]:
df.to_excel("scraping1.xlsx")

# retrieve data from mongodb

In [ ]:
# Get the first saved website as an example
a_website = database.websites.find_one()

In [ ]:
# Print the HTML code of the first domain and the first page
print(a_website['content']['1']['page'][:1000])